Credit: Some of the content in this script were taken from https://github.com/kennethban/bwf2020/tree/master/02%20-%20Alignment%2C%20Variant%20Calling%2C%20and%20Annotation

# Variant Calling

## Overview of variant calling workflow:

<img src="images/pipeline.png"/>

<center> Bolser, DM. A Simple SNP Calling Pipeline. Cited from https://www.ebi.ac.uk/sites/ebi.ac.uk/files/content.ebi.ac.uk/materials/2014/140217_AgriOmics/dan_bolser_snp_calling.pdf </center>


## Sequences collection and QC check

(We will skip this process, Direct to Sequences Trimming)

### Download the raw fastq files

We can download the raw fastq file on SRA website under Bioproject PRJNA606794 https://www.ncbi.nlm.nih.gov/sra?linkname=bioproject_sra_all&from_uid=606794

Download the raw fastq files using SRA-toolkit https://ncbi.github.io/sra-tools/fastq-dump.html

In [ ]:
fastq-dump --split-files <SRR-acession-number> -O <path-to-output-directory>

Example:

In [ ]:
fastq-dump --split-files SRR12165154 -O Fika/variant_calling/BRCA3/

This script will give paired-end fastq files:
SRR14149065_1.fastq
SRR14149065_2.fastq

### Download the human reference genome sequence

We can download the human reference genome sequences in https://asia.ensembl.org/Homo_sapiens/Info/Index

Let us look at the files that will be used

- `Homo_sapiens.GRCh38.dna.primary_assembly.fa` - the human reference genome (hg38)
- `SRR14149065_1.fastq` and `SRR14149065_2.fastq' - the sample sequences

### Taking a peek at the reference fasta file

In [ ]:
head /home/ref/Homo_sapiens.GRCh38.dna.primary_assembly.fa

The fasta format is quite simple. The first line is the identifier which starts with '>'

The subsequent lines are DNA sequences.

### Taking a look at the query sequence file (first 4 lines)

#### One sequence in a fastq file consists of 4 lines. 

- Line 1 - sequence identifier (starts with @)
- Line 2 - DNA sequence
- Line 3 - sequence identifier (starts with +)
- Line 4 - corresponding quality score (Phred score 0-93 + 33)

For the quality score, the following characters encode the lowest to highest scores

<pre> !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ </pre>

For more information, see https://en.wikipedia.org/wiki/FASTQ_format


### Checking the quality of the FASTQ sequences

It is a good practice to check the quality of the sequences by plotting the quality (Q) scores by the position. In general, a Q score of > 30 is good.

To generate a plot, we will use `fastQC` 

- Install on your laptop. Download on: (https://www.bioinformatics.babraham.ac.uk/projects/fastqc/)

- The analysis modules can be seen on https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/
- Example of good data: https://www.bioinformatics.babraham.ac.uk/projects/fastqc/good_sequence_short_fastqc.html
- Example of bad data: https://www.bioinformatics.babraham.ac.uk/projects/fastqc/bad_sequence_fastqc.html

## Trimming the fastq sequences

Trimmomatic is a fast, multithreaded command line tool that can be used to trim and crop Illumina (FASTQ) data as well as to remove adapters. These adapters can pose a real problem depending on the library preparation and downstream application.

There are two major modes of the program: Paired end mode and Single end mode. The paired end mode will maintain correspondence of read pairs and also use the additional information contained in paired reads to better find adapter or PCR primer fragments introduced by the library preparation process.

Trimmomatic works with FASTQ files, Files compressed using either „gzip‟ or „bzip2‟ are supported, and are identified by use of „.gz‟ or „.bz2‟ file extensions.

Trimmomatic performs a variety of useful trimming tasks for illumina paired-end and single ended data. The selection of trimming steps and their associated parameters are supplied on the command line.

The current trimming steps are:
- ILLUMINACLIP: Cut adapter and other illumina-specific sequences from the read. SLIDINGWINDOW: Performs a sliding window trimming approach. It starts scanning at the 5‟ end and clips the read once the average quality within the window falls below a threshold.
- MAXINFO: An adaptive quality trimmer which balances read length and error rate to maximise the value of each read
- LEADING: Cut bases off the start of a read, if below a threshold quality
- TRAILING: Cut bases off the end of a read, if below a threshold quality
- CROP: Cut the read to a specified length by removing bases from the end HEADCROP: Cut the specified number of bases from the start of the read MINLEN: Drop the read if it is below a specified length
- AVGQUAL: Drop the read if the average quality is below the specified level

For more information, you can go to: http://www.usadellab.org/cms/?page=trimmomatic

### Paired End Mode

For paired-end data, two input files, and 4 output files are specified, 2 for the 'paired' output where both reads survived the processing, and 2 for corresponding 'unpaired' output where a read survived, but the partner read did not.

Command for paired-end trimming:

java -jar <path-to-file-trimmomatic-0.35.jar> PE -phred33 <path-to-file-input_R1.fq.gz> <path-to-file-input_R2.fq.gz> <path-to-file-output_1P.fq.gz> <path-to-file-output_1U.fq.gz> <path-to-file-output_2P.fq.gz> <path-to-file-output_2U.fq.gz> (set-of-trimming-parameters)

Example for variant calling:

java -jar Trimmomatic-0.39/trimmomatic-0.39.jar PE -phred33 /home/data_latihan/variant_calling/BRCA_3_CMNH_17_1P.fastq.gz /home/data_latihan/variant_calling/BRCA_3_CMNH_17_2P.fastq.gz Fika/variant_calling/BRCA3/BRCA_3_1P.fastq Fika/variant_calling/BRCA3/BRCA_3_1U.fastq Fika/variant_calling/BRCA3/BRCA_3_2P.fastq Fika/variant_calling/BRCA3/BRCA_3_2U.fastq LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:50

Example for WGS:

java -jar Trimmomatic-0.39/trimmomatic-0.39.jar PE -phred33 /home/data_latihan/wgs/wgs/sample1_S1_L001_R1_001.fastq.gz /home/data_latihan/wgs/wgs/sample1_S1_L001_R2_001.fastq.gz Fika/wgs/sample1/sample1_1P.fastq Fika/wgs/sample1/sample1_1U.fastq Fika/wgs/sample1/sample1_2P.fastq Fika/wgs/sample1/sample1_2U.fastq LEADING:3 TRAILING:3 SLIDINGWINDOW:4:18 MINLEN:50